In [2]:
# Imports

import struct
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms
from torch.optim.lr_scheduler import StepLR
from PIL import Image

In [6]:
# Load IDX Data Function
def read_idx(filename):
    import struct
    import numpy as np
    with open(filename, 'rb') as f:
        magic, size = struct.unpack(">II", f.read(8))
        if magic == 2051:  # Images
            rows, cols = struct.unpack(">II", f.read(8))
            data = np.frombuffer(f.read(), dtype=np.uint8).reshape(size, rows, cols)
        elif magic == 2049:  # Labels
            data = np.frombuffer(f.read(), dtype=np.uint8)
        else:
            raise ValueError("Invalid IDX file format")
    return data

# Paths to IDX data
train_images_path = "/kaggle/input/mnist-dataset/train-images.idx3-ubyte"
train_labels_path = "/kaggle/input/mnist-dataset/train-labels.idx1-ubyte"
test_images_path = "/kaggle/input/mnist-dataset/t10k-images.idx3-ubyte"
test_labels_path = "/kaggle/input/mnist-dataset/t10k-labels.idx1-ubyte"

# Load data
train_images = read_idx(train_images_path)
train_labels = read_idx(train_labels_path)
test_images = read_idx(test_images_path)
test_labels = read_idx(test_labels_path)

# Normalize and convert to tensors
train_images_tensor = torch.tensor(train_images, dtype=torch.float32).unsqueeze(1) / 255.0
train_labels_tensor = torch.tensor(train_labels, dtype=torch.long)
test_images_tensor = torch.tensor(test_images, dtype=torch.float32).unsqueeze(1) / 255.0
test_labels_tensor = torch.tensor(test_labels, dtype=torch.long)

# Data Augmentation
from torchvision import transforms
train_transforms = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomAffine(0, translate=(0.1, 0.1)),
    transforms.ToTensor()
])

# Augmented Dataset Class
class AugmentedDataset(torch.utils.data.Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.images[idx].numpy()
        label = self.labels[idx].item()
        image = transforms.ToPILImage()(image)
        if self.transform:
            image = self.transform(image)
        return image, label

# Create datasets
train_dataset = AugmentedDataset(train_images_tensor, train_labels_tensor, transform=train_transforms)
test_dataset = TensorDataset(test_images_tensor, test_labels_tensor)

# Data loaders
from torch.utils.data import DataLoader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/mnist-dataset/train-images.idx3-ubyte'

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/mnist-dataset/train-images.idx3-ubyte'